In [1]:
import io
import os
import sys
import arcpy
import laspy
from zipfile import ZipFile
import zipfile
import pandas as pd
import requests
import ftplib
from bs4 import BeautifulSoup as bs
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureCollection, FeatureSet
from arcpy import env
from arcpy.sa import *

In [75]:
# Define the base URL and domain
DOMAIN = "https://resources.gisdata.mn.gov"
BASE_URL = "https://resources.gisdata.mn.gov/pub/data/elevation/lidar/county"
FILETYPE = ".laz"

# Directory where the .laz files will be saved
save_folder = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM"
os.makedirs(save_folder, exist_ok=True)  # Create the folder if it doesn't exist

In [81]:
# Dictionary mapping each county to its list of specific file numbers
county_file_map = {
    "winona": ["4342-30-62", "4342-30-63", "4342-30-64", "4342-31-62", "4342-31-63", "4342-31-64", 
               "4342-32-62", "4342-32-63", "4342-32-64", "4342-28-63", "4342-28-64", "4342-29-63", "4342-29-64"],
    "wabasha": ["4342-28-60", "4342-28-61", "4342-28-62", "4342-29-60", "4342-29-61", "4342-29-62"],
    "olmsted": ["4342-30-60", "4342-30-61", "4342-31-60", "4342-31-61", "4342-32-60", "4342-32-61"],
}

# Function to fetch and parse the HTML from the URL
def get_laz(url):
    return bs(requests.get(url).text, 'html.parser')

# Loop through each county in the dictionary
for county, file_numbers in county_file_map.items():
    # Construct the county-specific URL
    county_url = f"{BASE_URL}/{county}/laz/"
    
    # Loop through all links on the page and download the ones that match the file numbers
    for link in get_laz(county_url).find_all('a'):
        laz_link = link.get('href')
        if laz_link and FILETYPE in laz_link:  # Check if laz_link is not None and ends with .laz
            
            # Check if the file number is in the laz_link (assuming the file number is part of the filename)
            if any(file_number in laz_link for file_number in file_numbers):
                
                # Construct a full URL using county_url if relative path
                if laz_link.startswith('/'):
                    full_url = DOMAIN + laz_link  # Relative link, add DOMAIN
                else:
                    full_url = county_url + laz_link  # Absolute link, use it as is

                filename = os.path.basename(laz_link)  # Extract the filename from the URL
                
                # Full path to save the .laz file
                file_path = os.path.join(save_folder, filename)
                
                # Download and save the .laz file
                print(f"Downloading {full_url} from {county}...")
                try:
                    response = requests.get(full_url)
                    response.raise_for_status()  # Check for any errors
                    
                    with open(file_path, 'wb') as file:
                        file.write(response.content)
                    print(f"Saved to {file_path}")
                except requests.exceptions.RequestException as e:
                    print(f"Failed to download {full_url}: {e}")

Saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM\4342-28-63.laz
Saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM\4342-28-64.laz
Saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM\4342-29-63.laz
Saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM\4342-29-64.laz
Saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM\4342-30-62.laz
Saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM\4342-30-63.laz
Saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM\4342-30-64.laz
Saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM\4342-31-62.laz
Saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM\4342-31-63.laz
Saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM\4342-31-64.laz
Saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM\4342-32-62.laz
Saved to C:\Users\ethan\Desktop\ARLT_MGIS\G

In [82]:
# Specify the folder containing LAZ files and output paths
input_laz_folder = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\MN_laz_DEM"  # Path to your folder with .laz files
output_las_folder = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_files"  # Output folder for .las files
lasd_output = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_dataset.lasd"  # Output .lasd file

# Define the spatial reference using EPSG:26915 (NAD83 / UTM zone 15N)
spatial_ref = arcpy.SpatialReference(26915)  # EPSG:26915 for NAD83 / UTM zone 15N

# Ensure the output LAS folder exists
if not os.path.exists(output_las_folder):
    os.makedirs(output_las_folder)

# Remove existing LAS dataset if it exists
if os.path.exists(lasd_output):
    arcpy.management.Delete(lasd_output)  # Delete the existing LAS dataset to avoid errors

# Loop through all .laz files in the input folder
for laz_file in os.listdir(input_laz_folder):
    if laz_file.endswith(".laz"):
        input_laz_file = os.path.join(input_laz_folder, laz_file)

        # Convert LAZ to LAS using ConvertLas tool
        arcpy.conversion.ConvertLas(
            input_laz_file, 
            output_las_folder,  # Specify the target folder for .las files
            compression="NO_COMPRESSION",  # Disable compression
            define_coordinate_system="ALL_FILES",  # Define coordinate system for all files
            in_coordinate_system=spatial_ref  # Set the input coordinate system to EPSG:26915
        )
        print(f"Converted {laz_file} and added to {output_las_folder}")

# Create a LAS dataset (.lasd) from the LAS files and apply the spatial reference EPSG:26915
las_files = [os.path.join(output_las_folder, f) for f in os.listdir(output_las_folder) if f.endswith(".las")]

# Check if there are any LAS files to create a LAS dataset
if las_files:
    arcpy.management.CreateLasDataset(
        las_files,
        lasd_output,
        spatial_reference=spatial_ref,  # Apply EPSG:26915 spatial reference
        folder_recursion="NO_RECURSION",
        in_surface_constraints=""
    )
    print(f"LAS dataset created at {lasd_output} with EPSG:26915")
else:
    print("No LAS files found for creating LAS dataset.")

print("Process complete!")

Converted 4342-28-60.laz and added to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_files
Converted 4342-28-61.laz and added to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_files
Converted 4342-28-62.laz and added to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_files
Converted 4342-28-63.laz and added to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_files
Converted 4342-28-64.laz and added to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_files
Converted 4342-29-60.laz and added to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_files
Converted 4342-29-61.laz and added to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_files
Converted 4342-29-62.laz and added to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_files
Converted 4342-29-63.laz and added to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_files
Converted 4342-29-64.laz and added to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_files


In [83]:
# Define the path to the LAS dataset
lasd_file_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_dataset.lasd"

# Check if the LAS dataset exists
if os.path.exists(lasd_file_path):
    print(f"Retrieving statistics for LAS dataset at {lasd_file_path}...")

    try:
        # Use Describe to get the properties of the LAS dataset
        desc = arcpy.Describe(lasd_file_path)

        # Display the statistics
        print("LAS Dataset Statistics:")
        print(f"Point Count: {desc.pointCount}")
        print(f"Bounding Box: {desc.extent.XMin}, {desc.extent.YMin}, {desc.extent.XMax}, {desc.extent.YMax}")
        print(f"Z Min: {desc.zMin}")
        print(f"Z Max: {desc.zMax}")
        print(f"Average Return Number: {desc.avgReturnNumber}")
        print(f"Number of Returns: {desc.numberOfReturns}")

    except Exception as e:
        print(f"Failed to retrieve LAS dataset statistics: {e}")
else:
    print("LAS dataset not found.")

Retrieving statistics for LAS dataset at C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_dataset.lasd...
LAS Dataset Statistics:
Point Count: 202137958
Bounding Box: 567455.96, 4872214.87, 580161.41, 4889709.41
Failed to retrieve LAS dataset statistics: DescribeData: Method zMin does not exist


In [84]:
# Convert the .lasd to a DEM

# Paths to input LAS dataset and output DEM
las_dataset = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\las_dataset.lasd"  # Your LAS dataset file
output_dem = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\DEM\output_dem.tif"  # Output DEM (raster)

# Ensure the output DEM directory exists
output_dir = os.path.dirname(output_dem)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Convert LAS dataset to Raster (DEM) using Binning interpolation
arcpy.conversion.LasDatasetToRaster(
    las_dataset,
    output_dem,
    value_field="ELEVATION",  # Use ELEVATION field for DEM creation
    data_type="FLOAT",  # Set output raster data type to FLOAT
    z_factor=1,  # No vertical exaggeration
    )

print(f"DEM successfully created at: {output_dem}")

DEM successfully created at: C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\DEM\output_dem.tif


In [85]:
# Set the input DEM
input_dem = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\DEM\output_dem.tif"

# Set the output slope raster
output_slope = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\DEM\slope.tif"

# Ensure the Spatial Analyst extension is available
arcpy.CheckOutExtension("Spatial")

# Ensure the output directory for the slope exists
output_dir = os.path.dirname(output_slope)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Check if the input DEM exists
if not os.path.exists(input_dem):
    print(f"Input DEM not found at {input_dem}")
else:
    print("Input DEM exists.")

    # Convert input DEM to a raster object
    dem_raster = arcpy.Raster(input_dem)

    # Execute the Slope tool using the raster object
    slope_raster = Slope(dem_raster)

    # Save the output slope raster
    slope_raster.save(output_slope)

    print(f"Slope raster created successfully at {output_slope}!")

Input DEM exists.
Slope raster created successfully at C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\DEM\slope.tif!


### Landcover Data

In [92]:
# Inport Land cover data
from tqdm import tqdm  

# Define the URL and the output path for the ZIP file
url = r"https://s3-us-west-2.amazonaws.com/mrlc/nlcd_2021_land_cover_l48_20230630.zip"
zip_ncld_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\ncld_2021.zip"

# Download the file using requests.get(), stream=True for chunked downloading
response = requests.get(url, stream=True)

# Get the file size from headers (optional, for progress indication)
file_size = int(response.headers.get('content-length', 0))
print(f"Downloading NLCD ZIP file ({file_size / 1024 / 1024:.2f} MB)...")

# Stream the download and show a progress bar
with open(zip_ncld_path, 'wb') as f:
    with tqdm(total=file_size, unit='B', unit_scale=True, desc="Downloading") as pbar:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:  # Filter out keep-alive new chunks
                f.write(chunk)
                pbar.update(len(chunk))

# Open the ZIP file and print its contents
with zipfile.ZipFile(zip_ncld_path, 'r') as zip_ref:
    # Print the contents of the ZIP file
    zip_ref.printdir()

    # Create a directory for extraction if it doesn't exist
    extraction_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\Extracted"
    os.makedirs(extraction_path, exist_ok=True)

    # Extract all files to the specified directory
    zip_ref.extractall(extraction_path)

# Show the files you just extracted
print("Files in the extracted directory:")
print(os.listdir(extraction_path))

Downloading: 100%|██████████| 1.96G/1.96G [07:37<00:00, 4.28MB/s]﻿


File Name                                             Modified             Size
nlcd_2021_land_cover_l48_20230630.ige          2023-07-10 13:24:40  26350989969
nlcd_2021_land_cover_l48_20230630.img          2023-07-10 15:35:52    435940485
nlcd_2021_land_cover_l48_20230630.xml          2023-07-11 14:56:44        48467
Files in the extracted directory:
['nlcd_2021_land_cover_l48_20230630.ige', 'nlcd_2021_land_cover_l48_20230630.img', 'nlcd_2021_land_cover_l48_20230630.xml']


In [113]:
# Project the Raster
arcpy.management.ProjectRaster(
    in_raster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\Part2\Extracted\nlcd_2021_land_cover_l48_20230630.img",
    out_raster=r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\nlcd_2021_ProjectRaster",
    out_coor_system='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    resampling_type="NEAREST",
    cell_size="29.9521028000161 29.9882891646674",
    geographic_transform="WGS_1984_(ITRF00)_To_NAD_1983",
    Registration_Point=None,
    in_coor_system='PROJCS["Albers_Conical_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0]]',
    vertical="NO_VERTICAL"
)

<Result 'C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2_Part2\\GIS5571_Lab2_Part2.gdb\\nlcd_2021_ProjectRaster'>

In [116]:
# Confirm Projection
desc = arcpy.Describe(in_raster)
print(f"Raster projection: {desc.spatialReference.name}")

Raster projection: NAD_1983_UTM_Zone_15N


In [119]:
# Set workspace and output GDB
input_folder = r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2"
output_gdb = r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb"
clip_extent = "567455.96 4872214.87 580161.41 4889709.41"
nodata_value = "0"
year = "2021"  # Set the year or other variables
in_raster = r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\nlcd_2021_ProjectRaster"
template_dataset = None  # If needed, set this to a valid template dataset

# Define the output raster name
out_raster = os.path.join(output_gdb, f"nlcd_{year}_Clip")

# Debugging step to check if input raster exists
if arcpy.Exists(in_raster):
    print(f"Input raster exists: {in_raster}")
else:
    print("Input raster does not exist. Check path.")
    
# Perform the clipping operation
try:
    arcpy.management.Clip(
        in_raster=in_raster,
        rectangle=clip_extent,
        out_raster=out_raster,
        in_template_dataset=template_dataset,
        nodata_value=nodata_value,
        clipping_geometry="NONE",
        maintain_clipping_extent="MAINTAIN_EXTENT"
    )
    print(f"Clipped raster created at {out_raster}")
    
    # Check if output raster was created
    if arcpy.Exists(out_raster):
        print(f"Clipped raster exists: {out_raster}")
    else:
        print("Clipped raster was not created.")
        
except Exception as e:
    print(f"Failed to clip raster: {e}")

Input raster exists: C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\nlcd_2021_ProjectRaster
Clipped raster created at C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\nlcd_2021_Clip
Clipped raster exists: C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\nlcd_2021_Clip


In [111]:
# Confirm the Projection
projected_raster = os.path.join(output_gdb, "projected_raster")
arcpy.management.ProjectRaster(
    in_raster, 
    projected_raster, 
    out_coor_system=arcpy.SpatialReference(4326)  # Example: WGS 84
)

Raster Spatial Reference: Albers_Conical_Equal_Area


In [127]:
# Set the workspace and output geodatabase
output_gdb = r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb"
arcpy.env.workspace = output_gdb  # Set the environment workspace to the output GDB

# Get the raster for reclassification
in_raster = r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\nlcd_2021_Clip"

# Check if 'NLCD_Reclass' field exists, if not, add it
fields = [field.name for field in arcpy.ListFields(in_raster)]
reclass_field = "NLCD_Reclass"

if reclass_field not in fields:
    print(f"Field '{reclass_field}' does not exist. Adding it now...")
    
    # Adding the field to the raster's attribute table
    try:
        arcpy.management.AddField(
            in_table=in_raster,
            field_name=reclass_field,
            field_type="SHORT",  # Use a suitable type; SHORT is for integers
            field_alias="NLCD Reclassified"
        )
        print(f"Field '{reclass_field}' added successfully.")
        
        # Update the field based on NLCD values using an UpdateCursor
        with arcpy.da.UpdateCursor(in_raster, ["Value", reclass_field]) as cursor:
            for row in cursor:
                nlcd_value = row[0]
                if nlcd_value in [11]:
                    row[1] = 5
                elif nlcd_value in [21, 22, 23, 24, 31]:
                    row[1] = 4
                elif nlcd_value in [41, 42, 43, 52, 71]:
                    row[1] = 2
                elif nlcd_value in [81, 82]:
                    row[1] = 3
                elif nlcd_value in [90, 95]:
                    row[1] = 1
                else:
                    row[1] = 0  # For undefined or missing values
                cursor.updateRow(row)
        print(f"Field '{reclass_field}' populated based on NLCD values.")
    
    except arcpy.ExecuteError as e:
        print(f"Error adding or updating field: {arcpy.GetMessages(2)}")
    except Exception as e:
        print(f"An error occurred: {e}")

else:
    print(f"Field '{reclass_field}' already exists, proceeding with reclassification...")

# Perform the reclassification
try:
    out_raster = arcpy.sa.Reclassify(
        in_raster=in_raster,
        reclass_field=reclass_field,  # Reclassifying based on the new field
        remap="11 5;21 4;22 4;23 4;24 4;31 4;41 2;42 2;43 2;52 2;71 2;81 3;82 3;90 1;95 1",  # Adjust remap as needed
        missing_values="DATA"
    )

    # Save the reclassified raster
    output_name = os.path.join(output_gdb, f"ncld_{year}_reclass")
    out_raster.save(output_name)

    # Build pyramids for the new raster
    arcpy.management.BuildPyramids(output_name)

    print(f"Reclassification and pyramid building completed for: {output_name}")

except arcpy.ExecuteError as e:
    print(f"Failed to reclassify the raster: {arcpy.GetMessages(2)}")
except Exception as e:
    print(f"An error occurred: {e}")

Field 'NLCD_Reclass' does not exist. Adding it now...
Field 'NLCD_Reclass' added successfully.
Field 'NLCD_Reclass' populated based on NLCD values.
Reclassification and pyramid building completed for: C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\ncld_2021_reclass


In [4]:
# Reclass for AgLand Cost Raster

with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster="nlcd_2021_Clip",
        reclass_field="NLCD_Reclass",
        remap="1 6;2 4;3 10;4 1;5 8",
        missing_values="DATA"
    )
    out_raster.save(r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\AgCost")

In [5]:
# Reclass for Water Cost Raster

with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster="nlcd_2021_Clip",
        reclass_field="NLCD_Reclass",
        remap="1 6;2 2;3 3;4 1;5 8",
        missing_values="DATA"
    )
    out_raster.save(r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\WaterCost")

In [6]:
# Reclass for Slope Cost Raster

arcpy.ddd.Reclassify(
    in_raster="slope_raster",
    reclass_field="VALUE",
    remap="0 4 1;4 9 2;9 15 3;15 20 4;20 23 5;23 26 6;26 30 7;30 33 8;33 35 9;35 36 10",
    out_raster=r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\SlopeCost",
    missing_values="DATA"
)

<Result 'C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2_Part2\\GIS5571_Lab2_Part2.gdb\\SlopeCost'>

### Weighted Cost Path

#### Create Weighted Raster Layer with Cost Data
Use the cost rasters to create a weighted overlay for determining least cost path.

In [7]:
# Weighted Overlay
# Trial 1 using weights: slope = 50%, water = 30%, and ag = 20%
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb"):
    out_raster = arcpy.sa.WeightedOverlay(
        in_weighted_overlay_table=r"('C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\SlopeCost' 50 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; 10 1; 36 1; 37 1; 38 1; 39 1; 40 1; 41 1; 42 1; 43 1; 44 1; 45 1; 46 1; 47 1; 48 1; 49 1; 50 1; 51 1; 52 1; 53 1; 54 1; 55 1; 56 1; 57 1; 58 1; 59 1; 60 1; 61 1; 62 1; 63 1; 64 1; 65 1; 66 1; 67 1; 68 1; 69 1; 70 1; 71 1; 72 1; 73 1; 74 1; 75 1; 76 1; 77 1; 78 1; 79 1; NODATA NODATA); 'C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\WaterCost' 30 'Value' (1 1; 2 2; 3 3; 6 6; 8 8; NODATA NODATA); 'C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\AgCost' 20 'Value' (1 1; 4 4; 6 6; 8 8; 10 1; NODATA NODATA));1 9 1"
    )
    out_raster.save(r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\Trail1_Cost")

In [8]:
# Weighted Overlay
# Trial 2 using weights: slope = 60%, water = 30%, and ag = 10%
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb"):
    out_raster = arcpy.sa.WeightedOverlay(
        in_weighted_overlay_table=r"('C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\SlopeCost' 60 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; 10 1; 36 1; 37 1; 38 1; 39 1; 40 1; 41 1; 42 1; 43 1; 44 1; 45 1; 46 1; 47 1; 48 1; 49 1; 50 1; 51 1; 52 1; 53 1; 54 1; 55 1; 56 1; 57 1; 58 1; 59 1; 60 1; 61 1; 62 1; 63 1; 64 1; 65 1; 66 1; 67 1; 68 1; 69 1; 70 1; 71 1; 72 1; 73 1; 74 1; 75 1; 76 1; 77 1; 78 1; 79 1; NODATA NODATA); 'C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\WaterCost' 30 'Value' (1 1; 2 2; 3 3; 6 6; 8 8; NODATA NODATA); 'C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\AgCost' 10 'Value' (1 1; 4 4; 6 6; 8 8; 10 1; NODATA NODATA));1 9 1"
    )
    out_raster.save(r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\Trial2_Cost")

In [9]:
# Weighted Overlay
# Trial 3 using weights: slope = 33%, water = 34%, and ag = 33%
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb"):
    out_raster = arcpy.sa.WeightedOverlay(
        in_weighted_overlay_table=r"('C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\SlopeCost' 33 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; 10 1; 36 1; 37 1; 38 1; 39 1; 40 1; 41 1; 42 1; 43 1; 44 1; 45 1; 46 1; 47 1; 48 1; 49 1; 50 1; 51 1; 52 1; 53 1; 54 1; 55 1; 56 1; 57 1; 58 1; 59 1; 60 1; 61 1; 62 1; 63 1; 64 1; 65 1; 66 1; 67 1; 68 1; 69 1; 70 1; 71 1; 72 1; 73 1; 74 1; 75 1; 76 1; 77 1; 78 1; 79 1; NODATA NODATA); 'C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\WaterCost' 34 'Value' (1 1; 2 2; 3 3; 6 6; 8 8; NODATA NODATA); 'C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\AgCost' 33 'Value' (1 1; 4 4; 6 6; 8 8; 10 1; NODATA NODATA));1 9 1"
    )
    out_raster.save(r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\Trial3_Cost")

#### Least Cost Path Analysis
Use the Least Cost Path analysis tool to determine path variations with different weighting schemes applied to the cost layers.

In [10]:
#Trial 1 Least Cost Path

arcpy.intelligence.LeastCostPath(
    in_cost_surface="Trial1",
    in_start_point="Least Cost Path Input Starting Point (Points)",
    in_end_point="Least Cost Path Input Ending Point (Points)",
    out_path_feature_class=r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\Trial1_LeastCostPath",
    handle_zeros="SMALL_POSITIVE"
)

<Result 'C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2_Part2\\GIS5571_Lab2_Part2.gdb\\Trial1_LeastCostPath'>

In [11]:
#Trial 2 Least Cost Path

arcpy.intelligence.LeastCostPath(
    in_cost_surface="Trial2",
    in_start_point="Least Cost Path Input Starting Point (Points)",
    in_end_point="Least Cost Path Input Ending Point (Points)",
    out_path_feature_class=r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\Trial2_LeastCostPath",
    handle_zeros="SMALL_POSITIVE"
)

<Result 'C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2_Part2\\GIS5571_Lab2_Part2.gdb\\Trial2_LeastCostPath'>

In [12]:
#Trial 3 Least Cost Path

arcpy.intelligence.LeastCostPath(
    in_cost_surface="Trial3",
    in_start_point="Least Cost Path Input Starting Point (Points)",
    in_end_point="Least Cost Path Input Ending Point (Points)",
    out_path_feature_class=r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2_Part2\GIS5571_Lab2_Part2.gdb\Trial3_LeastCostPath",
    handle_zeros="SMALL_POSITIVE"
)

<Result 'C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2_Part2\\GIS5571_Lab2_Part2.gdb\\Trial3_LeastCostPath'>